In [ ]:
### MODULE FOR INPUT & THRESHOLD DEFINITION
# Library Imports for Image Processing and Analysis
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence
import cv2
import tifffile
import time
import os
import glob

# Input TIF Image Directory Required
InputDirectory = "Example/" # Directory
BaseFileName = "Example_" # File Name Prefix (Drug, Therapy, etc)
FileSuffix = "1" # File Name Suffix (Image replicate number)
FileFormat = ".tif" # File Format (tif format)

# Threshold Input
Trs1 = 10 # Blue channel threshold

Trs2 = 30 # Green channel threshold

Trs3 = 0 # Red channel threshold

# Handle Determination (RAW Intensity (1) or Thresholded Intensity (2))
Handle=2

# File Directory Checking
if not os.path.exists(InputDirectory):
    raise FileNotFoundError(f"The directory {InputDirectory} does not exist.")
elif not os.path.exists(f"{InputDirectory}/{BaseFileName}{FileSuffix}{FileFormat}"):
    raise FileNotFoundError(f"The file {BaseFileName}{FileSuffix}{FileFormat} does not exist.")
else:
    print(f"{InputDirectory}/{BaseFileName}{FileSuffix}{FileFormat} would be analyzed")


Drug Testing ICC/Example//Example_1.tif would be analyzed


In [217]:
## MODULE FOR REGION OF INTEREST (ROI) DETECTION / DESIGNATION

start_time = time.time()

# Input & Output Designation
data_output_path = InputDirectory + BaseFileName + FileSuffix + " HOUGH RESULT.csv" # Output
tiff_path = InputDirectory + BaseFileName + FileSuffix + FileFormat # Input

# Open TIFF and Channel Splitting
im = Image.open(tiff_path)
print(f"TIF image {tiff_path} opened") # TIFF Opening

page_list = [np.array(page.copy()) for page in ImageSequence.Iterator(im)] # Extract all slices from the TIFF stack
z_max = len(page_list) # Get the number of slices

blue_slices = np.array([page_list[i] for i in range(0, z_max, 3)])   # Every 3rd slice starting from 0 (blue)
green_slices = np.array([page_list[i] for i in range(1, z_max, 3)]) # Every 3rd slice starting from 1 (green)
red_slices = np.array([page_list[i] for i in range(2, z_max, 3)])  # Every 3rd slice starting from 2 (red)

print(f"Extracted {len(blue_slices)} blue slices, {len(green_slices)} green slices, {len(red_slices)} red slices.") # Check the results

# Save the split TIF images
blue_tif_path = f"{InputDirectory}{BaseFileName}{FileSuffix}_Blue_Channel.tif"
red_tif_path = f"{InputDirectory}{BaseFileName}{FileSuffix}_Red_Channel.tif"
green_tif_path = f"{InputDirectory}{BaseFileName}{FileSuffix}_Green_Channel.tif"

tifffile.imwrite(blue_tif_path, blue_slices)
tifffile.imwrite(green_tif_path, green_slices)
tifffile.imwrite(red_tif_path, red_slices)

print(f"Saved Blue channel to {blue_tif_path}")
print(f"Saved Green channel to {green_tif_path}")
print(f"Saved Red channel to {red_tif_path}")

# Circle Detection from middle of Green Channel
green_mid = (len(green_slices) // 2)
g_img = np.array(green_slices[green_mid])

print(f"Using slice {green_mid} out of {len(green_slices)} total slices (Middle of Green Channel)")

blurred = cv2.GaussianBlur(g_img, (7, 7), 5) # Apply Gaussian blur

circles = cv2.HoughCircles(
    blurred, cv2.HOUGH_GRADIENT, 1, 100, param1=12, param2=10, minRadius=40, maxRadius=75
) # Detect circles

circle_data = [] # Store circle data

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        x, y, radius = i[0], i[1], i[2]
        # Discard circles smaller than 65
        if radius >= 65:
            circle_data.append([x, y, radius])
            # Draw circles
            center = (x, y)
            cv2.circle(g_img, center, 1, (0, 100, 100), 3)  # Circle center
            cv2.circle(g_img, center, radius, (255, 0, 255), 2)  # Circle outline

# Save detected circles image
output_image_path = f"{InputDirectory}{BaseFileName}{FileSuffix}.png"
cv2.imwrite(output_image_path, g_img)
print(f"Saved circled image to {output_image_path}")

# Save to CSV
df = pd.DataFrame(circle_data, columns=["X", "Y", "Radius"])
df.to_csv(data_output_path, index=False)
print(f"Saved {len(circle_data)} detected circles to {data_output_path}")

end_time = time.time()
print(f"Time taken for detect_roi.py: {end_time - start_time} seconds")

TIF image Drug Testing ICC/Example/Example_1.tif opened
Extracted 18 blue slices, 18 green slices, 18 red slices.
Saved Blue channel to Drug Testing ICC/Example/Example_1_Blue_Channel.tif
Saved Green channel to Drug Testing ICC/Example/Example_1_Green_Channel.tif
Saved Red channel to Drug Testing ICC/Example/Example_1_Red_Channel.tif
Using slice 9 out of 18 total slices (Middle of Green Channel)
Saved circled image to Drug Testing ICC/Example/Example_1.png
Saved 40 detected circles to Drug Testing ICC/Example/Example_1 HOUGH RESULT.csv
Time taken for detect_roi.py: 0.28375244140625 seconds


In [218]:
## MODLUE FOR ROI SAVING INTO INDIVIDUAL TIFs

start_time = time.time()

# ROI Processing / Saving Function
def process_rois_and_save_tifs(BTIF, RTIF, GTIF, roi_data_path, output_folder, um_per_pixel):
    roi_df = pd.read_csv(roi_data_path) # Load the ROI data
    
    Blue_Total = tifffile.imread(BTIF) # Load the blue, red, and green channel images
    Red_Total = tifffile.imread(RTIF)
    Green_Total = tifffile.imread(GTIF)

    # Check if the images have the same dimensions
    if Blue_Total.shape != Red_Total.shape or Blue_Total.shape != Green_Total.shape:
        raise ValueError("The dimensions of the input TIFF images do not match.")

    os.makedirs(output_folder, exist_ok=True) # Create output folder if it doesn't exist
    
    files = glob.glob(os.path.join(output_folder, '*')) # Empty the output folder if it exists
    for f in files:
        os.remove(f)

    # Process each ROI
    saved_roi_count = 0
    for _, row in roi_df.iterrows():
        roi_x = row['X']
        roi_y = row['Y']
        roi_radius = row['Radius']
        roi_bounds = [roi_x - roi_radius, roi_y - roi_radius, roi_x + roi_radius, roi_y + roi_radius]
        roi = {'vnRectBounds': roi_bounds}

        A = roi['vnRectBounds'] # Read ROI bounds

        ay, by = max(0, A[0]), min(Blue_Total.shape[1], A[2]) # Define region of interest (ROI)
        ax, bx = max(0, A[1]), min(Blue_Total.shape[2], A[3])

        # Extract ROI for each channel
        roi_blue = Blue_Total[:, int(ax):int(bx), int(ay):int(by)]
        roi_red = Red_Total[:, int(ax):int(bx), int(ay):int(by)]
        roi_green = Green_Total[:, int(ax):int(bx), int(ay):int(by)]

        # Calculate average intensity in the middle sphere of 20 um diameter
        sphere_radius_px = int((20 / 2) / um_per_pixel)
        center_x = (bx - ax) // 2
        center_y = (by - ay) // 2
        center_z = roi_blue.shape[0] // 2

        z, y, x = np.ogrid[:roi_blue.shape[0], :roi_blue.shape[1], :roi_blue.shape[2]]
        mask = (z - center_z)**2 + (y - center_y)**2 + (x - center_x)**2 <= sphere_radius_px**2

        # Calculate green signal intensity within the sphere
        sphere_green_intensity = roi_green[mask]
        average_green_intensity = np.mean(sphere_green_intensity)

        # Check if green signal exceeds threshold
        if average_green_intensity > 5:
            print(f"ROI {saved_roi_count + 1}: High green signal intensity detected, might not have spheroid.")
            continue

        # Calculate blue average intensity within the sphere
        sphere_intensity = roi_blue[mask]
        average_intensity = np.mean(sphere_intensity)

        # Check if blue intensity is below threshold
        if average_intensity < 3:
            print(f"ROI {saved_roi_count + 1}: Average intensity below threshold, might not have spheroid.")
            continue

        saved_roi_count += 1 # Increment saved ROI count

        roi_stack = np.stack([roi_blue, roi_red, roi_green], axis=0) # Stack channels together

        # Save the ROI as a separate TIF file with zero-padded filename
        output_tif_path = os.path.join(output_folder, f"ROI{saved_roi_count:04d}.tif")
        tifffile.imwrite(output_tif_path, roi_stack.astype(np.uint16))
        print(f"Saved ROI {saved_roi_count} to {output_tif_path}")

# Define the input and output files
BTIF = InputDirectory + BaseFileName + FileSuffix + "_Blue_Channel.tif"
RTIF = InputDirectory + BaseFileName + FileSuffix + "_Red_Channel.tif"
GTIF = InputDirectory + BaseFileName + FileSuffix + "_Green_Channel.tif"
roi_data_path =  InputDirectory + BaseFileName + FileSuffix + " HOUGH RESULT.csv"
output_folder = InputDirectory + "/ROIs" + "_" + FileSuffix
um_per_pixel = 1.354
# Process ROIs and save TIFs
process_rois_and_save_tifs(BTIF, RTIF, GTIF, roi_data_path, output_folder, um_per_pixel)

end_time = time.time()
print(f"Time taken for detect_roi.py: {end_time - start_time} seconds")

Saved ROI 1 to Drug Testing ICC/Example//ROIs_1\ROI0001.tif
Saved ROI 2 to Drug Testing ICC/Example//ROIs_1\ROI0002.tif
Saved ROI 3 to Drug Testing ICC/Example//ROIs_1\ROI0003.tif
Saved ROI 4 to Drug Testing ICC/Example//ROIs_1\ROI0004.tif
Saved ROI 5 to Drug Testing ICC/Example//ROIs_1\ROI0005.tif
Saved ROI 6 to Drug Testing ICC/Example//ROIs_1\ROI0006.tif
Saved ROI 7 to Drug Testing ICC/Example//ROIs_1\ROI0007.tif
Saved ROI 8 to Drug Testing ICC/Example//ROIs_1\ROI0008.tif
Saved ROI 9 to Drug Testing ICC/Example//ROIs_1\ROI0009.tif
Saved ROI 10 to Drug Testing ICC/Example//ROIs_1\ROI0010.tif
Saved ROI 11 to Drug Testing ICC/Example//ROIs_1\ROI0011.tif
Saved ROI 12 to Drug Testing ICC/Example//ROIs_1\ROI0012.tif
Saved ROI 13 to Drug Testing ICC/Example//ROIs_1\ROI0013.tif
Saved ROI 14 to Drug Testing ICC/Example//ROIs_1\ROI0014.tif
Saved ROI 15 to Drug Testing ICC/Example//ROIs_1\ROI0015.tif
Saved ROI 16 to Drug Testing ICC/Example//ROIs_1\ROI0016.tif
Saved ROI 17 to Drug Testing ICC/

In [219]:
## MODULE FOR CENTROID DETERMINATION

# Splits the input interleaved TIFF image into red, green, and blue channels.
def split_tif_layers(tif_image):
    tif_image = np.array(tif_image)
    num_layers = tif_image.shape[0] // 3
    Blue_Total = tif_image[:num_layers]
    Red_Total = tif_image[num_layers:2*num_layers]
    Green_Total = tif_image[2*num_layers:]
    return Blue_Total, Green_Total, Red_Total

# Centroid and Radius Calculation
def calculate_centroid_and_radius(Blue_Trs, Z_Max_Input=None):
    Dimension = Blue_Trs.shape
    Max_Z = np.argmax(np.sum(Blue_Trs, axis=(0, 2, 3)))-4
    if Z_Max_Input is not None:
        Max_Z = Z_Max_Input
    ay, by = 1, Dimension[3]
    ax, bx = 1, Dimension[2]

    FI_BR_X, FI_BR_Y, Total_Intensity = 0.0, 0.0, 0.0
    Blue_Trs2 = Blue_Trs / 255
    for j in range(ay, by):
        for i in range(ax, bx):
            intensity = Blue_Trs2[0, Max_Z, i, j]
            if intensity > 0:
                FI_BR_X += i * intensity
                FI_BR_Y += j * intensity
                Total_Intensity += intensity

    # Debug: Check total intensity and centroid calculation
    print(f"Intermediate centroid calculation: FI_BR_X={FI_BR_X}, FI_BR_Y={FI_BR_Y}, Total_Intensity={Total_Intensity}")

    if Total_Intensity == 0:
        print("Skipped due to zero total intensity.")
        return None

    spheroid_radius = 1.354 * (Total_Intensity / 3.141592)**0.5
    Spheroid_Center_Coor = [
        round(FI_BR_X / Total_Intensity),
        round(FI_BR_Y / Total_Intensity),
        round(Max_Z)
    ]
    return Spheroid_Center_Coor, spheroid_radius

# Process ROIs to Determine Centroid
def process_saved_rois_and_calculate_centroids(rois_folder, Z_Max_Input=None):
    rois_files = sorted(glob.glob(os.path.join(rois_folder, 'ROI*.tif')))
    centroids_and_radii = []

    for roi_file in rois_files:
        roi_filename = os.path.basename(roi_file)
        roi_stack = tifffile.imread(roi_file)
        Blue_Total, Green_Total, Red_Total = split_tif_layers(roi_stack)

        Blue_Trs = (Blue_Total >= Trs1).astype(np.uint8) * 255
        Red_Trs = (Red_Total >= Trs2).astype(np.uint8) * 255
        Blue_Red_Trs = np.maximum(Blue_Trs, Red_Trs)

        # Debug: Check binary mask creation
        #print(f"{roi_filename} - Blue pixel count: {np.sum(Blue_Trs > 0)}, Red pixel count: {np.sum(Red_Trs > 0)}")

        result = calculate_centroid_and_radius(Blue_Red_Trs, Z_Max_Input)
        if result is not None:
            centroid, radius = result
            if centroid[2] >= 1 and centroid[2] <= roi_stack.shape[1] - 1:
                centroids_and_radii.append([roi_filename, *centroid, radius])
                print(f"Centroid for {roi_filename}: {centroid}, Radius: {radius:.2f}")
            else:
                print(f"{roi_filename} skipped due to invalid Z-coordinate: {centroid[2]}")

    return centroids_and_radii

# Output Processing Function
def save_centroids_and_radii_to_csv(centroids_and_radii, output_file):
    df = pd.DataFrame(
        centroids_and_radii,
        columns=['ROI#', 'X coordinate', 'Y coordinate', 'Z coordinate', 'Radius (um)']
    )
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    
    start_time = time.time()

    rois_folder = InputDirectory + "/ROIs" + "_" + FileSuffix
    Z_Max_Input = None

    centroids_and_radii = process_saved_rois_and_calculate_centroids(rois_folder, Z_Max_Input)
    output_file = os.path.join(InputDirectory, BaseFileName + FileSuffix + " centroids_and_radii.csv")
    save_centroids_and_radii_to_csv(centroids_and_radii, output_file)

    end_time = time.time()
    print(f"Time taken for detect_roi.py: {end_time - start_time} seconds")

Intermediate centroid calculation: FI_BR_X=870866.0, FI_BR_Y=624294.0, Total_Intensity=11197.0
Centroid for ROI0001.tif: [78, 56, 4], Radius: 80.83
Intermediate centroid calculation: FI_BR_X=471967.0, FI_BR_Y=524080.0, Total_Intensity=7464.0
Centroid for ROI0002.tif: [63, 70, 9], Radius: 66.00
Intermediate centroid calculation: FI_BR_X=520857.0, FI_BR_Y=569294.0, Total_Intensity=6959.0
Centroid for ROI0003.tif: [75, 82, 2], Radius: 63.73
Intermediate centroid calculation: FI_BR_X=697560.0, FI_BR_Y=660246.0, Total_Intensity=8319.0
Centroid for ROI0004.tif: [84, 79, 6], Radius: 69.68
Intermediate centroid calculation: FI_BR_X=979288.0, FI_BR_Y=1080333.0, Total_Intensity=14977.0
Centroid for ROI0005.tif: [65, 72, 8], Radius: 93.49
Intermediate centroid calculation: FI_BR_X=490127.0, FI_BR_Y=580901.0, Total_Intensity=7543.0
Centroid for ROI0006.tif: [65, 77, 3], Radius: 66.35
Intermediate centroid calculation: FI_BR_X=809700.0, FI_BR_Y=781321.0, Total_Intensity=10023.0
Centroid for ROI0007

In [220]:
## Calculate / REARRANGE SPATIAL FLUORESCENCE INTENSITY (FI) 

def calculate_fi(Blue_Input, Red_Input, Green_Input, Blue_Red_Input, Spheroid_Center_Coor, Radii, Pixel_to_um, ax, bx, ay, by, Dimension):
    # Set radii with 2.5 intervals
    radii_array = np.arange(0, 150, 2.5)
    FIraw = np.zeros((len(radii_array), 5))  # Initialize FI with columns: radius, Blue mean, Red mean, Green mean, Blue_Red mean
    FIraw[:, 0] = radii_array  # Set radius values
    count = np.zeros(len(radii_array))  # Count for normalization
    sumIntensities = np.zeros((len(radii_array), 4))  # Sum of intensities for average

    # Create a meshgrid for indices
    z, x, y = np.meshgrid(
        np.arange(-(Spheroid_Center_Coor[2]), (Dimension[1]) - Spheroid_Center_Coor[2]),
        np.arange((ax - Spheroid_Center_Coor[0]), (bx - Spheroid_Center_Coor[0])),
        np.arange((ay - Spheroid_Center_Coor[1]), (by - Spheroid_Center_Coor[1])),
        indexing='ij'
    )
    coords = np.stack([Pixel_to_um[1] * y, Pixel_to_um[0] * x, Pixel_to_um[2] * z], axis=1)

    # Distance calculation
    distances = np.linalg.norm(coords, axis=1)
    distances = (distances / 2.5).astype(int) * 2.5  # Adjust quantization to 2.5 intervals
    distances = np.expand_dims(np.transpose(distances, (0, 2, 1)), axis=0)

    # Flatten the intensity arrays
    Blue_Input_flat = Blue_Input
    Red_Input_flat = Red_Input
    Green_Input_flat = Green_Input
    Blue_Red_Input_flat = Blue_Red_Input
    print(f"Mask:{distances.shape}")
    print(f"Shapes - Blue: {Blue_Input_flat.shape}, Red: {Red_Input_flat.shape}, Green: {Green_Input_flat.shape}, Blue_Red: {Blue_Red_Input_flat.shape}, distances: {distances.shape}")

    for radius in radii_array:
        mask = distances == radius
        if radius >= Radii[0] + 2.5:  # Ensure radius is within valid range
            continue
        else:
            idx = np.where(radii_array == radius)[0][0]
            sumIntensities[idx, :] += np.sum(np.vstack((Blue_Input_flat[mask], Red_Input_flat[mask], Green_Input_flat[mask], Blue_Red_Input_flat[mask])), axis=1)
            count[idx] += np.sum(mask)

    averages = sumIntensities / count[:, None]

    # Combine results
    FIraw = np.column_stack((radii_array, averages[:, 0], averages[:, 1], averages[:, 3], count))
    Via = radii_array
    with np.errstate(divide='ignore', invalid='ignore'):
        Viability = np.where(averages[:, 3] == 0, 0, 100 - (averages[:, 1] / averages[:, 3]) * 100)

    Via = np.column_stack((Via, Viability))
    return FIraw, Via

def main():
    # Define the input folder and parameters
    rois_folder = InputDirectory + "/ROIs" + "_" + FileSuffix
    centroids_file = os.path.join(InputDirectory,BaseFileName + FileSuffix + " centroids_and_radii.csv")
    
    Pixel_to_um = np.array([1.354, 1.354, 10])  # um/pxl in XYZ space

    # Read the centroids from the CSV file
    centroids_df = pd.read_csv(centroids_file)
    print(centroids_df)
    for index, row in centroids_df.iterrows():
        roi_filename = row['ROI#']
        roi_filepath = os.path.join(rois_folder, roi_filename)
        Spheroid_Center_Coor = np.array([row['X coordinate'], row['Y coordinate'], row['Z coordinate']])
        Radii = np.array([row['Radius (um)']])
        print(f"Processing {roi_filepath}...")

        # Check if the ROI TIFF file exists
        if not os.path.exists(roi_filepath):
            print(f"File not found: {roi_filepath}")
            continue

        # Load the ROI TIFF file
        roi_stack = tifffile.imread(roi_filepath)

        # Split the TIFF layers into blue, green, and red channels
        Dimension = roi_stack.shape
        Blue_Total, Green_Total, Red_Total = split_tif_layers(roi_stack)

        if Handle==1:
            Blue_Input = np.squeeze(Blue_Total, axis=0)
            Red_Input = np.squeeze(Red_Total, axis=0)
            Green_Input = np.squeeze(Green_Total, axis=0)
        else:
            Blue_Input = (Blue_Total >= Trs1).astype(np.uint8) * 255
            Red_Input = (Red_Total >= Trs2).astype(np.uint8) * 255
            Green_Input = (Green_Total >= Trs3).astype(np.uint8) * 255


        # Ensure the arrays have the same shape before applying np.maximum
        if Blue_Input.shape == Red_Input.shape:
            Blue_Red_Input = np.maximum(Blue_Input, Red_Input)
        else:
            print(f"Shape mismatch: Blue_Input shape: {Blue_Input.shape}, Red_Input shape: {Red_Input.shape}")
            continue

        # Define the region of interest (ROI)
        ax, bx, ay, by = 0, Dimension[3], 0, Dimension[2]
        print("spheroid centroid", Spheroid_Center_Coor, "in", Dimension)
        Max_Z = Dimension[1]

        # Calculate FI and viability
        FIraw, Via = calculate_fi(Blue_Input, Red_Input, Green_Input, Blue_Red_Input, Spheroid_Center_Coor, Radii, Pixel_to_um, ax, bx, ay, by, Dimension)

        # Save FIraw and Via to CSV files
        roi_name = os.path.splitext(roi_filename)[0]
        FIraw_df = pd.DataFrame(FIraw, columns=['Radius (um)', 'Blue Mean', 'Red Mean', 'Blue_Red Mean', 'N of ROI analyzed'])
        FIraw_df.to_csv(os.path.join(rois_folder, f"{roi_name}_FIraw.csv"), index=False)

        if Handle==2:
            Via_df = pd.DataFrame(Via, columns=['Radius (um)', 'Viability'])
            Via_df.to_csv(os.path.join(rois_folder, f"{roi_name}_Via.csv"), index=False)

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(f"Time taken for detect_roi.py: {end_time - start_time} seconds")

           ROI#  X coordinate  Y coordinate  Z coordinate  Radius (um)
0   ROI0001.tif            78            56             4    80.834101
1   ROI0002.tif            63            70             9    65.997820
2   ROI0003.tif            75            82             2    63.726078
3   ROI0004.tif            84            79             6    69.675378
4   ROI0005.tif            65            72             8    93.488069
5   ROI0006.tif            65            77             3    66.346165
6   ROI0007.tif            81            78            11    76.479077
7   ROI0008.tif            86            68            11    78.894107
8   ROI0009.tif            64            80             5    68.369126
9   ROI0010.tif            69            66             2    50.389299
10  ROI0011.tif            77            79             3    91.853960
11  ROI0012.tif            75            53             5    54.677160
12  ROI0013.tif            70            75             6    93.003047
13  RO

C:\Users\Hyunsu Jeon\AppData\Local\Temp\ipykernel_20744\3220456684.py:42: RuntimeWarning: invalid value encountered in divide
  averages = sumIntensities / count[:, None]


Mask:(1, 18, 138, 138)
Shapes - Blue: (1, 18, 138, 138), Red: (1, 18, 138, 138), Green: (1, 18, 138, 138), Blue_Red: (1, 18, 138, 138), distances: (1, 18, 138, 138)
Processing Drug Testing ICC/Example//ROIs_1\ROI0004.tif...
spheroid centroid [84 79  6] in (3, 18, 148, 148)
Mask:(1, 18, 148, 148)
Shapes - Blue: (1, 18, 148, 148), Red: (1, 18, 148, 148), Green: (1, 18, 148, 148), Blue_Red: (1, 18, 148, 148), distances: (1, 18, 148, 148)
Processing Drug Testing ICC/Example//ROIs_1\ROI0005.tif...
spheroid centroid [65 72  8] in (3, 18, 144, 144)
Mask:(1, 18, 144, 144)
Shapes - Blue: (1, 18, 144, 144), Red: (1, 18, 144, 144), Green: (1, 18, 144, 144), Blue_Red: (1, 18, 144, 144), distances: (1, 18, 144, 144)
Processing Drug Testing ICC/Example//ROIs_1\ROI0006.tif...
spheroid centroid [65 77  3] in (3, 18, 136, 136)
Mask:(1, 18, 136, 136)
Shapes - Blue: (1, 18, 136, 136), Red: (1, 18, 136, 136), Green: (1, 18, 136, 136), Blue_Red: (1, 18, 136, 136), distances: (1, 18, 136, 136)
Processing Dr

In [221]:
def merge_csv_files(rois_folder, roi_numbers):
    merged_fi_data = {}
    merged_viability_data = {}

    for roi_number in roi_numbers:
        fi_csv_path = os.path.join(rois_folder, f"{roi_number}_FIraw.csv")
        via_csv_path = os.path.join(rois_folder, f"{roi_number}_Via.csv")

        # Check if the FIraw CSV file exists
        if not os.path.exists(fi_csv_path):
            print(f"Skipping {fi_csv_path}: file not found.")
            continue

        # Check if the Via CSV file exists
        if Handle==2:
            if not os.path.exists(via_csv_path):
                print(f"Skipping {via_csv_path}: file not found.")
                continue

        # Process FIraw CSV file
        fi_df = pd.read_csv(fi_csv_path)
        for _, row in fi_df.iterrows():
            radius = row['Radius (um)']
            if radius not in merged_fi_data:
                merged_fi_data[radius] = {'Radius (um)': radius, 'Blue': [], 'Red': [], 'Green': []}
            merged_fi_data[radius]['Blue'].append(row['Blue Mean'])
            merged_fi_data[radius]['Red'].append(row['Red Mean'])

        # Process Via CSV file
        if Handle==2:
            via_df = pd.read_csv(via_csv_path)
            for _, row in via_df.iterrows():
                radius = row['Radius (um)']
                if radius not in merged_viability_data:
                    merged_viability_data[radius] = {'Radius (um)': radius, 'Viability': []}
                merged_viability_data[radius]['Viability'].append(row['Viability'])

    # Merge FI data into a DataFrame
    merged_fi_df = pd.DataFrame.from_dict(merged_fi_data, orient='index')
    merged_fi_df = pd.concat([
        merged_fi_df[['Radius (um)']],
        pd.DataFrame(merged_fi_df['Blue'].tolist(), index=merged_fi_df.index).add_prefix('Blue '),
        pd.DataFrame(merged_fi_df['Red'].tolist(), index=merged_fi_df.index).add_prefix('Red ')
    ], axis=1)

    # Merge Viability data into a DataFrame
    if Handle==2:
        merged_viability_df = pd.DataFrame.from_dict(merged_viability_data, orient='index')

    # Flatten the 'Viability' column into multiple columns
    if Handle==2:
        viability_columns = pd.DataFrame(merged_viability_df['Viability'].tolist(), index=merged_viability_df.index)

    # Rename all columns in the flattened DataFrame to "Viability"
    if Handle==2:
        viability_columns.columns = ['Viability'] * viability_columns.shape[1]  # Set "Viability" as the name for every column

    # Combine the 'Radius' column with the flattened viability data
    if Handle==2:
        merged_viability_df = pd.concat([
        merged_viability_df[['Radius (um)']],
        viability_columns
        ], axis=1)

    # Save the merged data to CSV files
    merged_fi_df.to_csv(os.path.join(InputDirectory, BaseFileName + FileSuffix + " Converged_FIraw.csv"), index=False)
    if Handle==2:
        merged_viability_df.to_csv(os.path.join(InputDirectory, BaseFileName + FileSuffix + " Converged_Via.csv"), index=False)

if __name__ == "__main__":
    start_time = time.time()
    rois_folder = InputDirectory + "ROIs" + "_" + FileSuffix
    
    centroids_file = os.path.join(InputDirectory,BaseFileName + FileSuffix + " centroids_and_radii.csv")
    centroids_df = pd.read_csv(centroids_file)
    print(centroids_df)

    roi_numbers = [f"ROI{str(index + 1).zfill(4)}" for index in range(len(centroids_df)*2)]
    print(f"ROI numbers to be processed: {roi_numbers}")
    merge_csv_files(rois_folder, roi_numbers)

    end_time = time.time()
    print(f"Time taken for detect_roi.py: {end_time - start_time} seconds")

           ROI#  X coordinate  Y coordinate  Z coordinate  Radius (um)
0   ROI0001.tif            78            56             4    80.834101
1   ROI0002.tif            63            70             9    65.997820
2   ROI0003.tif            75            82             2    63.726078
3   ROI0004.tif            84            79             6    69.675378
4   ROI0005.tif            65            72             8    93.488069
5   ROI0006.tif            65            77             3    66.346165
6   ROI0007.tif            81            78            11    76.479077
7   ROI0008.tif            86            68            11    78.894107
8   ROI0009.tif            64            80             5    68.369126
9   ROI0010.tif            69            66             2    50.389299
10  ROI0011.tif            77            79             3    91.853960
11  ROI0012.tif            75            53             5    54.677160
12  ROI0013.tif            70            75             6    93.003047
13  RO